In [ ]:
from datetime import datetime, timedelta
import os
import streamlit as st
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from fpdf import FPDF

In [ ]:
# LLM + Tools
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

In [ ]:
# API Keys
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [ ]:
# Initialize LLM and Search Tool
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
search = SerpAPIWrapper()
search_tool = Tool(
    name="Google Search",
    func=search.run,
    description="Search live stock news, comparisons, and prices."
)
agent = initialize_agent(
    tools=[search_tool],
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

In [ ]:
# Streamlit UI Setup
st.set_page_config(page_title="AI Stock Bot", layout="centered")
st.title("🌍 Multi-Country AI Stock Assistant")

In [ ]:
# Country Market Selector
suffix_map = {
    "India": ".NS",
    "USA": "",
    "Germany": ".DE",
    "UK": ".L",
    "France": ".PA",
    "Hong Kong": ".HK"
}
currency_map = {
    "India": "₹",
    "USA": "$",
    "Germany": "€",
    "UK": "£",
    "France": "€",
    "Hong Kong": "HK$"
}
ticker_hint = {
    "India": "e.g., INFY.NS",
    "USA": "e.g., AAPL",
    "Germany": "e.g., BMW.DE",
    "UK": "e.g., SHEL.L",
    "France": "e.g., AIR.PA",
    "Hong Kong": "e.g., 0700.HK"
}

country = st.selectbox("🌐 Select Market", list(suffix_map.keys()))
suffix = suffix_map[country]
currency = currency_map[country]

query_type = st.selectbox("📌 Choose an Action", [
    "Fetch Stock Price",
    "Compare Multiple Stocks",
    "Get Stock Market News",
    "Visualize Stock Chart",
    "Set Price Alert",
    "Export Stock Data",
    "Get Earnings & Financial Ratios"
])

def full_ticker(ticker):
    return ticker if ticker.endswith(suffix) else ticker + suffix

In [ ]:
# Country Market Selector
suffix_map = {
    "India": ".NS",
    "USA": "",
    "Germany": ".DE",
    "UK": ".L",
    "France": ".PA",
    "Hong Kong": ".HK"
}
currency_map = {
    "India": "₹",
    "USA": "$",
    "Germany": "€",
    "UK": "£",
    "France": "€",
    "Hong Kong": "HK$"
}
ticker_hint = {
    "India": "e.g., INFY.NS",
    "USA": "e.g., AAPL",
    "Germany": "e.g., BMW.DE",
    "UK": "e.g., SHEL.L",
    "France": "e.g., AIR.PA",
    "Hong Kong": "e.g., 0700.HK"
}

In [ ]:
# Fetch Price
if query_type == "Fetch Stock Price":
    stock = st.text_input("Enter Stock Ticker", placeholder=ticker_hint[country])
    if stock and st.button("Get Price"):
        data = yf.Ticker(full_ticker(stock)).history(period="1d")
        if not data.empty:
            price = round(data['Close'].iloc[-1], 2)
            st.success(f"📊 {stock.upper()} Current Price: {currency}{price}")
        else:
            st.error("Could not fetch price.")

In [ ]:
# Compare Stocks
elif query_type == "Compare Multiple Stocks":
    tickers = st.text_input("Comma-separated Tickers", placeholder=ticker_hint[country])
    if tickers and st.button("Compare"):
        stocks = [full_ticker(s.strip()) for s in tickers.split(",")]
        prices = {}
        for s in stocks:
            d = yf.Ticker(s).history(period="1d")
            prices[s] = round(d['Close'].iloc[-1], 2) if not d.empty else "N/A"
        st.dataframe(pd.DataFrame(prices.items(), columns=["Stock", f"Price ({currency})"]))

In [ ]:
# Stock Market News
elif query_type == "Get Stock Market News":
    company = st.text_input("Enter Company Name (e.g., Tesla, Infosys)")
    if company and st.button("Summarize News"):
        prompt = f"Summarize recent stock news and performance for {company}"
        summary = agent.run(prompt)
        st.info(summary)

In [ ]:
# Visualize Chart
elif query_type == "Visualize Stock Chart":
    stock = st.text_input("Enter Ticker", placeholder=ticker_hint[country])
    start_date = st.date_input("Start Date", datetime.today() - timedelta(days=30))
    end_date = st.date_input("End Date", datetime.today())
    if stock and st.button("Show Chart"):
        data = yf.download(full_ticker(stock), start=start_date, end=end_date)
        if not data.empty:
            fig, ax = plt.subplots(figsize=(10, 5))
            ax.plot(data.index, data["Close"], marker='o', linestyle='-')
            ax.set_title(f"{stock.upper()} – Closing Prices")
            ax.set_xlabel("Date")
            ax.set_ylabel(f"Price ({currency})")
            ax.grid(True)
            st.pyplot(fig)
        else:
            st.warning("Couldn't fetch historical data.")

In [ ]:
# Price Alert
elif query_type == "Set Price Alert":
    stock = st.text_input("Ticker", placeholder=ticker_hint[country])
    alert = st.number_input("Alert when price is below:", min_value=0.0)
    if stock and st.button("Check Now"):
        d = yf.Ticker(full_ticker(stock)).history(period="1d")
        if not d.empty:
            current = round(d['Close'].iloc[-1], 2)
            if current <= alert:
                st.warning(f"🚨 Alert! {stock.upper()} is at {currency}{current} — BELOW your alert!")
            else:
                st.info(f"✅ {stock.upper()} is at {currency}{current}. No alert triggered.")
        else:
            st.error("Could not retrieve data.")

In [ ]:
# Export Data
elif query_type == "Export Stock Data":
    symbol = st.text_input("Enter Ticker", placeholder=ticker_hint[country])
    period = st.selectbox("Select Period", ["1d", "5d", "1mo", "3mo", "6mo", "1y"])
    file_type = st.radio("Choose Format", ["CSV", "PDF"])
    if symbol and st.button("Export"):
        hist = yf.Ticker(full_ticker(symbol)).history(period=period)
        if hist.empty:
            st.error("No data available.")
        else:
            filename = f"{symbol}_{period}.{file_type.lower()}"
            if file_type == "CSV":
                hist.to_csv(filename)
            else:
                pdf = FPDF()
                pdf.add_page()
                pdf.set_font("Arial", size=12)
                for _, row in hist.reset_index().iterrows():
                    pdf.cell(200, 10, txt=f"{row['Date']} - {currency}{round(row['Close'],2)}", ln=True)
                pdf.output(filename)
            with open(filename, "rb") as f:
                st.download_button("📥 Download File", f, file_name=filename)

In [ ]:
# Earnings & Ratios
elif query_type == "Get Earnings & Financial Ratios":
    stock = st.text_input("Enter Ticker", placeholder=ticker_hint[country])
    if stock and st.button("Fetch Financials"):
        tkr = yf.Ticker(full_ticker(stock))
        info = tkr.info
        keys = ["forwardPE", "trailingPE", "pegRatio", "returnOnEquity", "returnOnAssets", "debtToEquity"]
        data = {k: info.get(k, "N/A") for k in keys}
        st.subheader("📊 Key Financial Ratios")
        st.json(data)
        try:
            st.subheader("📅 Earnings Calendar")
            st.json(tkr.calendar.to_dict())
        except Exception:
            st.write("Not available.")